# Aujourd'hui on se fait rouler par les mecs de l'ENS


https://challengedata.ens.fr/en/challenge/39/prediction_of_transaction_claims_status.html


Ici, c'est le notebook dédié uniquement à:
1. Gestion des données manquantes
2. Correction des erreurs

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

#load packages
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

import seaborn as sns
print("seaborn version: {}". format(sns.__version__))

#misc libraries
import time
import random as rnd
import os, gc

#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Moi j'ai ça:

Python version: 3.6.3 |Anaconda custom (64-bit)| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]

pandas version: 0.22.0

matplotlib version: 2.1.2

NumPy version: 1.12.1

SciPy version: 1.0.0

IPython version: 6.2.1

scikit-learn version: 0.19.1

seaborn version: 0.8.1


In [ ]:
# Run "! pip install jyquickhelper" dans une cellule si ca ne marche pas la commande suivante
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [ ]:
# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Définition de la seed pour le random

Très important pour qu'on voit les mêmes choses entre nos deux ordis

In [ ]:
RANDOM_SEED = 42;
np.random.seed(RANDOM_SEED)

## Chargement des données

In [ ]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
DATA_PATH = os.path.join(PROJECT_ROOT_DIR, "data")
DATA_PROCESSED = os.path.join(PROJECT_ROOT_DIR, "data_processed")

def load_data(file,data_path=DATA_PATH, sep=','):
    csv_path = os.path.join(data_path, file)
    return pd.read_csv(csv_path, sep)

TX_data = load_data(file = "input_train.csv");
STA_data = load_data(file = "challenge_output_data_training_file_prediction_of_transaction_claims_status.csv", sep=';');
TX_data = pd.merge(TX_data, STA_data, left_index=True, right_index=True) # Merge de TX_data et STA_data en utilisant les index comme cléf primaire


train_df=TX_data.drop(["ID_y","ID_x"],inplace=False,axis=1) # On drop les ID qui sont crée dans le processus

test_df = load_data(file = "input_test.csv");

del STA_data #Je supprime toujours les variables inutiles pour liberer ma RAM (qui est très faible)

## Analyse des types de données

In [ ]:
#train_df.info()

A l'état initial nous avons comme type de variables

**Variables Catégorielles**
1. `SHIPPING_MODE`
2. `SHIPPING_PRICE`
3. `WARRANTIES_PRICE`
4. `PRICECLUB_STATUS`
5. `PURCHASE_COUNT`
7. `BUYING_DATE`
8. `SELLER_SCORE_COUNT`
9. `SELLER_SCORE_AVERAGE`
10. `SELLER_COUNTRY`
11. `PRODUCT_TYPE`
12. `PRODUCT_FAMILY`
13. `ITEM_PRICE`
14. `CLAIM_TYPE` - La target

**Variables Booléennes**
1. `WARRANTIES_FLG`
2. `CARD_PAYMENT`
3. `COUPON_PAYMENT`
4. `RSP_PAYMENT`
5. `WALLET_PAYMENT`

**Variables Numériques Discrètes**
1. `REGISTRATION_DATE`
2. `BUYER_BIRTHDAY_DATE`
3. `BUYER_DEPARTMENT` - Que je propose de passer maitenent en catégorie, puisque les nombres n'ont pas de valeur + ou -
4. `SELLER_DEPARTMENT` - Que je propose de passer maitenent en catégorie, puisque les nombres n'ont pas de valeur + ou -

**Variables Numériques Continues**
 - Aucune

**Variables Ordinales**
 - Aucune

In [ ]:
train_df.BUYER_DEPARTMENT = train_df.BUYER_DEPARTMENT.astype('object')
train_df.SELLER_DEPARTMENT = train_df.SELLER_DEPARTMENT.astype('object')

test_df.BUYER_DEPARTMENT = test_df.BUYER_DEPARTMENT.astype('object')
test_df.SELLER_DEPARTMENT = test_df.SELLER_DEPARTMENT.astype('object')

In [ ]:
train_df.head()

**Variables Mixtes**

C'est à dire qu'elles contiennes en fait de l'alphanumérique, et des structures à l'interieur

1. `SHIPPING_PRICE`         - Format Nombre </> Nombre
2. `WARRANTIES_PRICE`         - Format Nombre </> Nombre
3. `PURCHASE_COUNT`         - Format Nombre </> Nombre
4. `BUYING_DATE`         - Format Nombre/2017
5. `SELLER_SCORE_COUNT`         - Format Nombre </> Nombre
6. `ITEM_PRICE`         - Format Nombre </> Nombre

**Données Manquantes**

1. `SHIPPING_MODE`           : 315
2. `SHIPPING_PRICE`          : 67610
3. `WARRANTIES_PRICE`        : 96603
4. `PRICECLUB_STATUS`        : 57
5. `BUYER_BIRTHDAY_DATE`     : 5836
6. `SELLER_SCORE_COUNT`      : 6
7. `SELLER_SCORE_AVERAGE`    : 6


**Données Avec Erreurs**

1. `BUYER_BIRTHDAY_DATE` - Des gens centenaires
2. `BUYER_DEPARTMENT` - Départements -1, 0, 97, et 98 
3. `SELLER_DEPARTMENT` - Départements avec -1
4. `PRODUCT_FAMILY` - des "White" qui ne veulent rien dire


**Dans l'ensemble**

- 16 variables catégorielles dont 6 mixtes
- 5 variables booléennes
- 2 numériques

- 8 ont des valeurs manquantes
- 4 ont des erreurs

In [ ]:
train_df.info()
print('_'*40)
test_df.info()

## Traiter les données manquantes

Valeurs à remplir :
1. `SHIPPING_MODE`  
2. `SHIPPING_PRICE` 
3. `WARRANTIES_PRICE` 
4. `PRICECLUB_STATUS`
5. `BUYER_BIRTHDAY_DATE` - Possède des erreurs, on va la traiter après
6. `SELLER_SCORE_COUNT`
7. `SELLER_SCORE_AVERAGE`

### SHIPPING_MODE

In [ ]:
train_df.SHIPPING_MODE.sort_values().unique()

In [ ]:
## SHIPPING_MODE
'''
Propose de remplacer les valeurs manquantes par "ABSENT". Je souhaite faire comprendre à l'algorithme qu'il n'y a pas l'info

Sinon, on peut remplacer par "NORMAL"
'''

train_df.SHIPPING_MODE.fillna(value="ABSENT", inplace=True)
test_df.SHIPPING_MODE.fillna(value="ABSENT", inplace=True)

### SHIPPING_PRICE

In [ ]:
train_df.SHIPPING_PRICE.sort_values().unique()

In [ ]:
## SHIPPING_PRICE
'''
Propose de remplacer les valeurs manquantes par "0". Puisque c'était gratuit

Sinon, on peut remplacer par "<1"
'''

train_df.SHIPPING_PRICE.fillna(value="0", inplace=True)
test_df.SHIPPING_PRICE.fillna(value="0", inplace=True)

### WARRANTIES_PRICE

In [ ]:
train_df.WARRANTIES_PRICE.sort_values().unique()

In [ ]:
## WARRANTIES_PRICE 
'''
Propose de remplacer les valeurs manquantes par "0". Puisque c'était gratuit

Sinon, on peut remplacer par "<1"
'''

train_df.WARRANTIES_PRICE.fillna(value="0", inplace=True)
test_df.WARRANTIES_PRICE.fillna(value="0", inplace=True)

### PRICECLUB_STATUS

In [ ]:
print(train_df.PRICECLUB_STATUS.sort_values().unique())

In [ ]:
## PRICECLUB_STATUS 
'''
Propose de remplacer les valeurs manquantes par "UNSUBSCRIBED". Puisque c'est par défaut

Sinon, on peut remplacer par "NONE" pour montrer qu'il manque l'info
'''

train_df.PRICECLUB_STATUS.fillna(value="UNSUBSCRIBED", inplace=True)
test_df.PRICECLUB_STATUS.fillna(value="UNSUBSCRIBED", inplace=True)

### SELLER_SCORE_COUNT

In [ ]:
train_df.SELLER_SCORE_COUNT.sort_values().unique()

In [ ]:
## SELLER_SCORE_COUNT 
'''
Propose de remplacer les valeurs manquantes par le mode de la série ('10000<100000').

Sinon, on peut remplacer par "-1" pour montrer qu'il manque l'info
'''

mode_train= train_df.SELLER_SCORE_COUNT.mode()[0]


train_df.SELLER_SCORE_COUNT.fillna(value=mode_train, inplace=True)
test_df.SELLER_SCORE_COUNT.fillna(value=mode_train, inplace=True)

### SELLER_SCORE_AVERAGE

In [ ]:
train_df.SELLER_SCORE_AVERAGE.sort_values().unique()

In [ ]:
## SELLER_SCORE_AVERAGE 
'''
Propose de remplacer les valeurs manquantes par la médiane de la série (46).

Sinon, on peut remplacer par "-1" pour montrer qu'il manque l'info
'''

mode_train= train_df.SELLER_SCORE_AVERAGE.median()


train_df.SELLER_SCORE_AVERAGE.fillna(value=mode_train, inplace=True)
test_df.SELLER_SCORE_AVERAGE.fillna(value=mode_train, inplace=True)

## Traiter les erreurs

Valeurs à corriger :

1. `BUYER_BIRTHDAY_DATE` - Des gens centenaires
2. `BUYER_DEPARTMENT` - Départements avec -1 et 0
3. `SELLER_DEPARTMENT` - Départements avec -1
4. `PRODUCT_FAMILY` - des "White" qui ne veulent rien dire

### BUYER_BIRTHDAY_DATE

In [ ]:
# Avant
sns.distplot(train_df.BUYER_BIRTHDAY_DATE.dropna(inplace=False));

In [ ]:
mu, sigma = train_df.BUYER_BIRTHDAY_DATE.mean(), train_df.BUYER_BIRTHDAY_DATE.std() # mean et standard deviation
print("Mu = {} et sigma = {}".format(mu, sigma))

In [ ]:
def normal_filler(value, mu, sigma, threshold):
    if np.isnan(value) or value<threshold:
        return np.random.normal(mu, sigma, 1)[0]
    else:
        return value;

In [ ]:
train_df.BUYER_BIRTHDAY_DATE = train_df.BUYER_BIRTHDAY_DATE.apply(normal_filler, args=(mu,sigma,1930))
test_df.BUYER_BIRTHDAY_DATE = test_df.BUYER_BIRTHDAY_DATE.apply(normal_filler, args=(mu,sigma,1930))

In [ ]:
# Après
sns.distplot(train_df.BUYER_BIRTHDAY_DATE);

### BUYER_DEPARTMENT

On doit remplacer les valeurs:
    -1, 0, 96, 97, 98 qui n'existent pas

In [ ]:
print("On a:\n{} valeurs à -1\n{} valeurs à 0\n{} valeurs à 96\n{} valeurs à 97\n{} valeurs à 98\n ".format(
train_df[train_df.BUYER_DEPARTMENT == -1].shape[0],
train_df[train_df.BUYER_DEPARTMENT == 0].shape[0],
train_df[train_df.BUYER_DEPARTMENT == 96].shape[0],
train_df[train_df.BUYER_DEPARTMENT == 97].shape[0],
train_df[train_df.BUYER_DEPARTMENT == 98].shape[0]))

In [ ]:
'''
Comme ca n'a pas de sens de remplir avec une loi normale
je propose de corriger tout ça en mettant tout à "0" pour montrer à l'algorithme le problème
''' 

def departement_filler(value):
    if value not in range(-1,96):
        return 0;
    else:
        return value;

train_df.BUYER_DEPARTMENT = train_df.BUYER_DEPARTMENT.apply(departement_filler)
test_df.BUYER_DEPARTMENT = test_df.BUYER_DEPARTMENT.apply(departement_filler)

### SELLER_DEPARTMENT

In [ ]:
print("On a:\n{} valeurs à -1\n{} valeurs à 0\n{} valeurs à 96\n{} valeurs à 97\n{} valeurs à 98\n ".format(
            train_df[train_df.SELLER_DEPARTMENT == -1].shape[0],
            train_df[train_df.SELLER_DEPARTMENT == 0].shape[0],
            train_df[train_df.SELLER_DEPARTMENT == 96].shape[0],
            train_df[train_df.SELLER_DEPARTMENT == 97].shape[0],
            train_df[train_df.SELLER_DEPARTMENT == 98].shape[0]
        )
     )

In [ ]:
'''
42 318 valeurs à -1 parce qu'a l'étranger, ça me semble impossible à gérer de façon smart

On remplace tout les autres à zéro? Oui? Très bien
''' 

train_df.SELLER_DEPARTMENT = train_df.SELLER_DEPARTMENT.apply(departement_filler)
test_df.SELLER_DEPARTMENT = test_df.SELLER_DEPARTMENT.apply(departement_filler)

### PRODUCT_FAMILY

In [ ]:
print("On a {} valeurs à WHITE".format(train_df[train_df.PRODUCT_FAMILY == 'WHITE'].shape[0]))

Un peu complexe là, on sait que WHITE contient une majorité de `PRODUCT_TYPE` "BRICOLAGE", donc bon, faudrait refaire un mapping dans la classe, mais j'ai la flemme

### Bilan des corrections d'erreurs et de données manquantes

Ben on est bien là! On a corrigé nos trucs, et tout le monde est content

In [ ]:
train_df.info()
print('_'*40)
test_df.info()

## Traitement des variables

**Variables Mixtes**

C'est à dire qu'elles contiennes en fait de l'alphanumérique, et des structures à l'interieur

1. `SHIPPING_PRICE`         - Format Nombre </> Nombre
2. `WARRANTIES_PRICE`         - Format Nombre </> Nombre
3. `PURCHASE_COUNT`         - Format Nombre </> Nombre
4. `BUYING_DATE`         - Format Nombre/2017
5. `SELLER_SCORE_COUNT`         - Format Nombre </> Nombre
6. `ITEM_PRICE`         - Format Nombre </> Nombre

Bon, en y regardant de plus près:

On ne peut pas tirer de ces variables catégorielles des variables continues.

La seule information que nous avons c'est l'ordre de grandeur. Alors transformons les en variables __ordinales__

### SHIPPING_PRICE

In [ ]:
# Exemple avant
print(train_df.SHIPPING_PRICE.sort_values().unique())
print(test_df.SHIPPING_PRICE.sort_values().unique())

In [ ]:
mapper= {'0':0, '<1':1, '1<5':2, '5<10':3, '10<20':4, '>20':5}

train_df.SHIPPING_PRICE = train_df.SHIPPING_PRICE.map(mapper)
test_df.SHIPPING_PRICE = test_df.SHIPPING_PRICE.map(mapper)

In [ ]:
# Et maintenant après transformation
print(train_df.SHIPPING_PRICE.sort_values().unique())
print(test_df.SHIPPING_PRICE.sort_values().unique())

### WARRANTIES_PRICE

In [ ]:
print(train_df.WARRANTIES_PRICE.sort_values().unique())
print(test_df.WARRANTIES_PRICE.sort_values().unique())

In [ ]:
mapper={'0':0, '<5':1, '5<20':2, '20<50':3, '50<100':4,  '100<500':5}

train_df.WARRANTIES_PRICE = train_df.WARRANTIES_PRICE.map(mapper)
test_df.WARRANTIES_PRICE = test_df.WARRANTIES_PRICE.map(mapper)

In [ ]:
print(train_df.WARRANTIES_PRICE.sort_values().unique())
print(test_df.WARRANTIES_PRICE.sort_values().unique())

### PURCHASE_COUNT

In [ ]:
print(train_df.PURCHASE_COUNT.sort_values().unique())
print(test_df.PURCHASE_COUNT.sort_values().unique())

In [ ]:
mapper={'<5':0, '5<20':1, '20<50':2, '50<100':3, '100<500':4,  '>500':5}

train_df.PURCHASE_COUNT = train_df.PURCHASE_COUNT.map(mapper)
test_df.PURCHASE_COUNT = test_df.PURCHASE_COUNT.map(mapper)

In [ ]:
print(train_df.PURCHASE_COUNT.sort_values().unique())
print(test_df.PURCHASE_COUNT.sort_values().unique())

### BUYING_DATE

In [ ]:
print(train_df.BUYING_DATE.sort_values().unique())
print(test_df.BUYING_DATE.sort_values().unique())

In [ ]:
toto= train_df.BUYING_DATE.str.split('/', 1, expand=True)
train_df.BUYING_DATE = toto[0]
train_df.BUYING_DATE = train_df.BUYING_DATE.apply(pd.to_numeric, errors='ignore')

toto= test_df.BUYING_DATE.str.split('/', 1, expand=True)
test_df.BUYING_DATE = toto[0]
test_df.BUYING_DATE = test_df.BUYING_DATE.apply(pd.to_numeric, errors='ignore')

del toto

In [ ]:
print(train_df.BUYING_DATE.sort_values().unique())
print(test_df.BUYING_DATE.sort_values().unique())

### SELLER_SCORE_COUNT

In [ ]:
print(train_df.SELLER_SCORE_COUNT.sort_values().unique())
print(test_df.SELLER_SCORE_COUNT.sort_values().unique())

In [ ]:
mapper={'<100':0, '100<1000':1, '1000<10000':2, '10000<100000':3, '100000<1000000':4}

train_df.SELLER_SCORE_COUNT = train_df.SELLER_SCORE_COUNT.map(mapper)
test_df.SELLER_SCORE_COUNT = test_df.SELLER_SCORE_COUNT.map(mapper)

In [ ]:
print(train_df.SELLER_SCORE_COUNT.sort_values().unique())
print(test_df.SELLER_SCORE_COUNT.sort_values().unique())

### ITEM_PRICE

In [ ]:
print(train_df.ITEM_PRICE.sort_values().unique())
print(test_df.ITEM_PRICE.sort_values().unique())

In [ ]:
mapper={'<10':0, '10<20':1, '20<50':2, '50<100':3, '100<500':4,  '500<1000':5, '1000<5000':6, '>5000':6} #Il existe qu'un >5000, et il n'existe pas dans le jeu de test

train_df.ITEM_PRICE = train_df.ITEM_PRICE.map(mapper)
test_df.ITEM_PRICE = test_df.ITEM_PRICE.map(mapper)

In [ ]:
print(train_df.ITEM_PRICE.sort_values().unique())
print(test_df.ITEM_PRICE.sort_values().unique())

### PRICECLUB_STATUS

In [ ]:
print(train_df.PRICECLUB_STATUS.sort_values().unique())
print(test_df.PRICECLUB_STATUS.sort_values().unique())

Le classement des types de comptes est inscrit ici : https://www.priceminister.com/evt/rakuten-superpoints

En faisant ça, je veux garder l'information "classement"

In [ ]:
mapper={'UNSUBSCRIBED':0, 'REGULAR':1, 'SILVER':2, 'GOLD':3, 'PLATINUM':4}

train_df.PRICECLUB_STATUS = train_df.PRICECLUB_STATUS.map(mapper)
test_df.PRICECLUB_STATUS = test_df.PRICECLUB_STATUS.map(mapper)

In [ ]:
print(train_df.PRICECLUB_STATUS.sort_values().unique())
print(test_df.PRICECLUB_STATUS.sort_values().unique())

## Drop Duplicates


Finding duplicates in your data is important because you end up:

1. Spending more computational time to process duplicates, which slows your algorithms down.

2. Obtaining false results because duplicates implicitly overweight the results. 

Because some entries appear more than once, the algorithm considers these entries more important.

Ok, mais ici, on a aucun duplicate dans la classe majoritaire, donc je préfère ne rien droper finalement

In [ ]:
train_df.shape

In [ ]:
train_df.drop_duplicates(inplace=False).shape

Mais si on enlève les duplicates on perd 3149 lignes... Donc c'est mort de mon point de vue

### Bilan

Ben on est bien là! On a transformé tout ça en variables __ordinales__, et tout le monde est content

In [ ]:
train_df.info()
print('_'*40)
test_df.info()

Actuellement, il nous reste quatre variables catégorielles:
1. `SHIPPING_MODE`
2. `SELLER_COUNTRY`
3. `PRODUCT_TYPE`
4. `PRODUCT_FAMILY`

Il n'y a pas de classement à faire entre ces catégories, on verra ça dans le Notebook 2 Feature Engineering.

Mais quel teasing!

Et on oublie pas d'écrire les résultats de nos transformations.

In [ ]:
filename_train = DATA_PROCESSED+"/train_notebook_1.csv"
filename_test = DATA_PROCESSED+"/test_notebook_1.csv"

try:
    os.remove(filename_train)
    os.remove(filename_test)
except:
    pass;
    
train_df.to_csv(filename_train, index=False, sep=";")
test_df.to_csv(filename_test, index=False, sep=";")